In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
from selenium import webdriver
import os

In [2]:
!which chromedriver

'which' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
executable_path = {'executable_path': '.\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA MARS NEWS

In [4]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html= browser.html
soup= bs(html, 'html.parser')
news_title = soup.find("div", class_='content_title').get_text()
news_p = soup.find('div', class_='article_teaser_body').get_text()
print(news_title)
print(news_p)

Mars Now
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


# JPL MARS SPACE IMAGES- FEATURED IMAGE

In [5]:
url_image="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)
browser.click_link_by_partial_text('FULL IMAGE')
expand = browser.find_by_css('a.fancybox-expand')

C:\Users\Colin Ek\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [6]:
html_image = browser.html
soup = bs(html_image, 'html.parser')
featured_image_url= soup.find("img",{"class":"fancybox-image"})["src"]
mainurl="https://www.jpl.nasa.gov"
original_image_url= mainurl + featured_image_url
print(original_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15256_ip.jpg


# Mars Facts

In [7]:
web_url= "https://space-facts.com/mars/"
browser.visit(web_url)
html_web=browser.html
soup=bs(html_web, "html.parser")

In [8]:
tab = pd.read_html(web_url)
# tab
mdf1=tab[0]
mdf2=tab[1]
mdf1.columns= ["Elements","Facts/Observations"]
mdf1.set_index("Elements", inplace= True)
html_table = mdf1.to_html()
# html_table

In [9]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Facts/Observations</th>    </tr>    <tr>      <th>Elements</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [10]:
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
html = browser.html
soup = bs(html, 'html.parser')
hemis = []
results = soup.find_all('div', class_="collapsible results")
hemispheres = results[0].find_all('h3')

for x in hemispheres:
    hemis.append(x.text)
hemis

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [11]:
thumbnail_results = results[0].find_all('a')
thumbnail_links = []

for thumbnail in thumbnail_results:
    if (thumbnail.img):
        thumbnail_url = 'https://astrogeology.usgs.gov/' + thumbnail['href']
        thumbnail_links.append(thumbnail_url)

thumbnail_links

['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

In [12]:
image_list = []
for url in thumbnail_links:
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    results = soup.find_all('img', class_='wide-image')
    image_path = results[0]['src']
    img_link = 'https://astrogeology.usgs.gov/' + image_path
    image_list.append(img_link)

image_list

['https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [13]:
hemisphere_zip = zip(hemis, image_list)
mars_urls = []

for title, i in hemisphere_zip:
    mars1 = {}
    mars1['title'] = title
    mars1['img_url'] = i
    mars_urls.append(mars1)

mars_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [15]:
# !pip install nbconvert
!jupyter nbconvert --to python Mission_to_Mars_Final.ipynb

[NbConvertApp] Converting notebook Mission_to_Mars_Final.ipynb to python
[NbConvertApp] Writing 2894 bytes to Mission_to_Mars_Final.py
